In [ ]:
%load_ext autoreload
%autoreload 2
%aimport

In [ ]:
import os 
import sys
path = os.path.abspath('../..')
if path not in sys.path: 
    sys.path.insert(0, path)
sys.path[0]

In [ ]:
import numpy as np

import matplotlib.pyplot as plt

import torch
import torch.optim as optim

import fitsio 

import json

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print('torch version: ', torch.__version__)
print(device)

In [ ]:
from src import psf_transform_lib
from src.utils import const
from src.data import simulated_datasets_lib
from src import starnet_lib

# Load 

In [ ]:
# this is the PSF I fitted using ground truth Hubble locations/fluxes. 
init_psf_params = torch.tensor(np.load('../../data/fitted_powerlaw_psf_params.npy'), device=device)
power_law_psf = psf_transform_lib.PowerLawPSF(init_psf_params)
psf = power_law_psf.forward().detach()

# number of bands. Here, there are two. 
n_bands = psf.shape[0]
print(n_bands)
print(psf.shape)


# Plot images

## test star simulator

In [ ]:
# data parameters
with open('../../data/default_star_parameters.json', 'r') as fp:
    data_params = json.load(fp)
data_params['max_stars'] = 10
data_params['mean_stars'] = 5
data_params['slen'] = 30 
print(data_params)

In [ ]:
# set background 
background = torch.zeros(n_bands, data_params['slen'], data_params['slen'], device=device)
background[0] = 686.
background[1] = 1123.

In [ ]:
# draw data 
n_images = 10

simulated_dataset = \
    simulated_datasets_lib.StarsDataset.load_dataset_from_params(n_images,
                    data_params, psf,
                    background,
                    transpose_psf = False, 
                    add_noise = True, draw_poisson=True)

In [ ]:
# test sampling params 
n_sources, locs, params = simulated_dataset.simulator.sample_parameters(batchsize=1)

print(n_sources.shape, locs.shape, params.shape)
print('n_sources:\n', n_sources)
print('locs:\n', locs)
print('params:\n', params)

# check psf 
print(simulated_dataset.simulator.psf.shape)

In [ ]:
locs[0][:,0]

In [ ]:
#drawing
images = simulated_dataset.simulator.draw_image_from_params(locs, n_sources, fluxes=params)
print(images.shape)
plt.matshow(images.cpu().numpy()[0,0])

locs_x = locs[0][:,0].cpu().numpy() * (simulated_dataset.slen-1)
locs_y = locs[0][:,1].cpu().numpy() * (simulated_dataset.slen-1)
plt.scatter(locs_y, locs_x, marker='x', color='red')

# Test star encoder

In [ ]:
star_encoder = starnet_lib.SourceEncoder(slen = 30,
                                            patch_slen = 8,
                                            step = 2,
                                            edge_padding = 3, 
                                            n_bands = 2,
                                            max_detections = 2)

In [ ]:
# data parameters
with open('../../data/default_star_parameters.json', 'r') as fp:
    data_params = json.load(fp)
data_params['max_stars'] = 10
data_params['mean_stars'] = 5
data_params['slen'] = 30 
print(data_params)

In [ ]:
# draw data 
n_images = 10

simulated_dataset = \
    simulated_datasets_lib.StarsDataset.load_dataset_from_params(n_images,
                    data_params, psf,
                    background,
                    transpose_psf = False, 
                    add_noise = True, draw_poisson=True)

# test sampling params 
n_sources, locs, fluxes = simulated_dataset.simulator.sample_parameters(batchsize=1)
images = simulated_dataset.simulator.draw_image_from_params(locs, n_sources, fluxes=params)
print(n_sources.shape, locs.shape, fluxes.shape, images.shape)

In [ ]:
image_patches, patch_locs, patch_fluxes, patch_n_stars, _ = star_encoder.get_image_patches(images, 
                                           locs = locs, 
                                           source_params= fluxes)

print(image_patches.shape, patch_locs.shape, patch_fluxes.shape, patch_n_stars.shape)

In [ ]:
#print where they are stars.. 
patch_n_stars.nonzero()

In [ ]:
# same plot as before, but I also mark my estimated star on this star
indx = 119
plt.matshow(image_patches[indx, 1].detach().cpu().numpy())

# the neural network returns locations / fluxes for any stars in the center of the grid
plt.axvline(x=star_encoder.edge_padding - 0.5, color = 'r')
plt.axvline(x=star_encoder.patch_slen - star_encoder.edge_padding - 0.5, color = 'r')
plt.axhline(y=star_encoder.edge_padding - 0.5, color = 'r')
plt.axhline(y=star_encoder.patch_slen - star_encoder.edge_padding - 0.5, color = 'r')


# lets mark any true sources in this particular tile 
if patch_n_sources[indx] > 0: 
    _loc = patch_locs[indx][0:patch_n_sources[indx]].detach().cpu()
    # you have to scale and shift appropriately so the location plots in the right place. 
    # all locations, whether on the full image or the tile, is parameterized to be between 0 and 1
    _loc = _loc * (star_encoder.patch_slen - 2 * star_encoder.edge_padding) +  star_encoder.edge_padding - 0.5
    plt.scatter(_loc[:, 1], _loc[:, 0], marker = 'o', color = 'b')
    
# plot my estimated star
if patch_n_sources[indx] > 0: 
    _loc = loc_mean[indx][0:patch_n_sources[indx]].detach().cpu()
    # you have to scale and shift appropriately so the location plots in the right place. 
    # all locations, whether on the full image or the tile, is parameterized to be between 0 and 1
    _loc = _loc * (star_encoder.patch_slen - 2 * star_encoder.edge_padding) +  star_encoder.edge_padding - 0.5
    plt.scatter(_loc[:, 1], _loc[:, 0], marker = 'x', color = 'r')


## Test galaxy simulation

In [ ]:
# data parameters
with open('../../data/default_galaxy_parameters.json', 'r') as fp:
    data_params = json.load(fp)
print(data_params)

In [ ]:
n_images=100
sim_ds = simulated_datasets_lib.GalaxyDataset.load_dataset_from_params(100, data_params)

In [ ]:
# background 
torch.mean(sim_ds.background[2])

In [ ]:
from torch.distributions import Normal
p_z = Normal(torch.zeros((1,), device=device), torch.ones((1,), device=device))
z = p_z.rsample(torch.tensor([2, 8])).view(2, -1)
print(z)

In [ ]:
#check that a couple of galaxies from the dataset look good. 
fig, axs = plt.subplots(3, 3, figsize=(15,15))
axes = axs.flatten()
for i, gal in enumerate(sim_ds.simulator.ds): 
    if i > 8: 
        break   
    ax = axes[i]
    im = ax.imshow(gal[2].cpu().numpy())
    fig.colorbar(im, ax=ax)

In [ ]:
from torch.distributions import Normal
p_z = Normal(torch.zeros(1), torch.ones(1))
z = p_z.rsample(torch.tensor([2, 8])).view(2, -1)  # shape = (8,)
print(z.shape)
z

In [ ]:
# test sampling works fine 
z, gals = sim_ds.simulator.ds.sample(2)
gals = gals.detach().cpu().numpy()
print(z.shape) 
print(gals.shape)
plt.imshow(gals[0][0])

In [ ]:
# test sampling source params
n_galaxy = torch.Tensor([3]).cuda(device)
galaxy_params, single_galaxies = sim_ds.simulator.get_source_params(n_galaxy)

print(galaxy_params.shape, single_galaxies.shape)
plt.imshow(single_galaxies[0][0][0].cpu().numpy())
assert single_galaxies[0, 3:, ...].sum() == 0  #all galaxies should be zero after. 
galaxy_params[0, 0:6, ...] # the rest should be zero after the third 


In [ ]:
# test sample parameters
n_sources, locs, params = sim_ds.simulator.sample_parameters(1)
gal_params, single_galaxies = params 
print(n_sources.shape, 
      locs.shape, 
      gal_params.shape, 
      single_galaxies.shape)

In [ ]:
#test image drawing
images = sim_ds.simulator.draw_image_from_params(locs, n_sources, sources=single_galaxies.cuda(device))
print(images.shape)

print(n_sources)
plt.figure(figsize=(12,12))
plt.imshow(images[0][0].detach().cpu().numpy(), interpolation='None')
plt.colorbar()

## Smaller galaxy scene

In [ ]:
# data parameters
with open('../../data/default_galaxy_parameters.json', 'r') as fp:
    data_params = json.load(fp)
data_params['min_galaxies'] = 1
data_params['mean_galaxies'] = 3 
data_params['max_galaxies'] = 5
print(data_params)

In [ ]:
n_images=100
sim_ds = simulated_datasets_lib.GalaxyDataset.load_dataset_from_params(100, data_params)

In [ ]:
#draw each of the images produced. 

In [ ]:
sample = sim_ds[0]
plt.figure(figsize=(10,10))
image =sample['image'][2].detach().cpu().numpy() 
plt.imshow(image)

# Encoder

## prepare the encoder and the simulated dataset 

In [ ]:
import src.starnet_lib as starnet_lib

In [ ]:
gal_dim = 8 # number of latent dimensions. 
galaxy_encoder = starnet_lib.SourceEncoder(
                            slen = 100, 
                            n_bands = 1,
                            # patches may need to be larger now
                            # and corresponding padding parameters should change
                            patch_slen = 20, 
                            step = 5, 
                            edge_padding = 5, 
                            max_detections = 2, 
                            is_star = False,
                            # the number of source parameters 
                            n_source_params = gal_dim).cuda()

In [ ]:
# get params and image 
n_images=100
# data parameters
with open('../../data/default_galaxy_parameters.json', 'r') as fp:
    data_params = json.load(fp)
data_params['min_galaxies'] = 1
data_params['mean_galaxies'] = 3
data_params['max_galaxies'] = 5
print(data_params)


sim_ds = simulated_datasets_lib.GalaxyDataset.load_dataset_from_params(100, data_params)

In [ ]:
for i, x in enumerate(sim_ds):
    
    print(x.keys())
    if i==0: 
        break 

## sample from dataset

In [ ]:
sample = sim_ds[0]
n_sources = sample['n_sources']
locs = sample['locs']
source_params = sample['source_params']
images = sample['images']
print(n_sources)

print(n_sources.shape, locs.shape, source_params.shape, images.shape)

In [ ]:
# draw the image with correct centers
plt.figure(figsize=(9,9))
plt.imshow(images[0,0].cpu().numpy())
locs_x = locs[0][:,0].cpu().numpy() * (sim_ds.slen-1)
locs_y = locs[0][:,1].cpu().numpy() * (sim_ds.slen-1)
plt.scatter(locs_y, locs_x, marker='x', color='red')

In [ ]:
%matplotlib inline

#compare with decoder drawing directly which should be in the center of the image? 
plt.figure(figsize=(6,6))
plt.imshow(sim_ds.simulator.ds[0][0].cpu().numpy())
print(sim_ds.simulator.galaxy_slen)
plt.scatter((sim_ds.simulator.galaxy_slen-1)/2, (sim_ds.simulator.galaxy_slen-1)/2, marker='x', color='r')

# so probably because I move it around the central pixel 

## patching

In [ ]:
image_patches, patch_locs, patch_galaxy_params, patch_n_sources, patch_is_on_array = \
    galaxy_encoder.get_image_patches(images, 
                                     locs = locs, 
                                      source_params = source_params)


print(image_patches.shape, patch_locs.shape, patch_galaxy_params.shape, patch_n_sources.shape)

In [ ]:
# in which patches are they sources' centers. 
patch_n_sources.nonzero()

In [ ]:
%matplotlib inline
#plot some patch
plt.matshow(image_patches[103][0].cpu().numpy())

## get variational parameters

In [ ]:
var_params_all = galaxy_encoder.get_var_params_all(image_patches)
var_params_all.shape
#why 63? 

In [ ]:
# get variational parameters for n_sources
log_probs_nsource_patch = galaxy_encoder.get_logprob_n_from_var_params(var_params_all)
log_probs_nsource_patch.shape 
#why 3  = 0,1,2 possible detections

In [ ]:
# get variational parameters at true number of stars 
loc_mean, loc_logvar, \
    galaxy_param_mean, galaxy_param_logvar = \
        galaxy_encoder.get_var_params_for_n_sources(var_params_all, 
                                        # we clip at max detections
                                        n_sources=patch_n_sources.clamp(max = galaxy_encoder.max_detections))

# galaxy_param_mean has shape n_patches x max_detections x gal_dim
print(loc_mean.shape, galaxy_param_mean.shape)
#

In [ ]:
loc_mean[2]
# if more than one galaxy was detected there would be more locations in the second row. 

## plot a tiles to check 

In [ ]:
plt.close('all')

In [ ]:
%matplotlib inline
fig, axs = plt.subplots(5,4,figsize=(17,17))
axes = axs.flatten()
# same plot as before, but I also mark my estimated galaxy on each patch
for i, indx in enumerate(patch_n_sources.nonzero()):
    if i >= len(axes): 
        break 
        
    indx = indx.item()
    n_band = 0
    ax = axes[i]
    ax.matshow(image_patches[indx, n_band].detach().cpu().numpy())

    # the neural network returns locations / fluxes for any stars in the center of the grid
    ax.axvline(x=galaxy_encoder.edge_padding - 0.5, color = 'r')
    ax.axvline(x=galaxy_encoder.patch_slen - galaxy_encoder.edge_padding - 0.5, color = 'r')
    ax.axhline(y=galaxy_encoder.edge_padding - 0.5, color = 'r')
    ax.axhline(y=galaxy_encoder.patch_slen - galaxy_encoder.edge_padding - 0.5, color = 'r')


    # lets mark any true sources in this particular tile 
    if patch_n_sources[indx] > 0: 
        _loc = patch_locs[indx][0:patch_n_sources[indx]].detach().cpu()
        # you have to scale and shift appropriately so the location plots in the right place. 
        # all locations, whether on the full image or the tile, is parameterized to be between 0 and 1
        _loc = _loc * (galaxy_encoder.patch_slen - 2 * galaxy_encoder.edge_padding) +  galaxy_encoder.edge_padding - 0.5
        ax.scatter(_loc[:, 1], _loc[:, 0], marker = 'o', color = 'b')

    # plot my estimated star
    if patch_n_sources[indx] > 0: 
        _loc = loc_mean[indx][0:patch_n_sources[indx]].detach().cpu()
        # you have to scale and shift appropriately so the location plots in the right place. 
        # all locations, whether on the full image or the tile, is parameterized to be between 0 and 1
        _loc = _loc * (galaxy_encoder.patch_slen - 2 * galaxy_encoder.edge_padding) +  galaxy_encoder.edge_padding - 0.5
        ax.scatter(_loc[:, 1], _loc[:, 0], marker = 'x', color = 'r')


# next step

# Testing

In [ ]:
is_on_array = const.get_is_on_from_n_sources(n_stars, 5)
is_on_array
# this means that they are only 5 stars in the first image. 

In [ ]:
(2 < n_stars).float()

In [ ]:
n=2
is_on_n = (n < n_stars).float()
locs[:, n, :] * is_on_n.unsqueeze(1)

In [ ]:
class Parent(object):
    def __init__(self, x): 
        self.x = x
        self.y = self.get_y()
        
    @staticmethod
    def get_y():
        return 3

class Child(Parent):
        
    @staticmethod
    def get_y(): 
        return 4 

In [ ]:
c = Child(3) 

In [ ]:
c.y